In [2]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [3]:
DATA = pd.read_csv('data.csv')
DATA['Demand'] = DATA['Demand']/1000

In [ ]:
DATA.index = pd.to_datetime(DATA['Time'], format='%Y-%m-%d %H:%M:%S')
train_set, test_set = DATA.loc[:'2018-12-31'], DATA.loc['2019-01-01':]
ntrain = len(train_set)
ntest = len(test_set)

In [ ]:
seconds = DATA.index.map(pd.Timestamp.timestamp)
day = 24*60*60
week = 7*day
year = (365.2425)*day
DATA['Day sin'] = np.sin(seconds * (2 * np.pi / day))
DATA['Day cos'] = np.cos(seconds * (2 * np.pi / day))
DATA['Week sin'] = np.sin(seconds * (2 * np.pi / week))
DATA['Week cos'] = np.cos(seconds * (2 * np.pi / week))
DATA['Year sin'] = np.sin(seconds * (2 * np.pi / year))
DATA['Year cos'] = np.cos(seconds * (2 * np.pi / year))

In [ ]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(DATA[['Demand','NonWeekend_holiday','Temperature',
                                    'Day sin','Day cos','Week sin','Week cos',
                                    'Year sin','Year cos']])

# shift regressors' series so the value at moment t is used for prediction at t
reg = scaled[1:,1:]
scaled[:(len(scaled)-1),1:] = reg


# create a proper input (X) tensor & output (y) vector 

X = []
y = []

horizon = 1     # forecast horizon
nlags = 24      # number of lags used for forecasting

for i in range(nlags, len(scaled)-horizon+1):
    X.append(scaled[i-nlags:i, :])
    y.append(scaled[i+horizon-1:i+horizon, 0])

X, y = np.array(X), np.array(y)

print(X.shape, y.shape)
xdim = X.shape[2]
X_train, y_train = X[:(ntrain-nlags)], y[:(ntrain-nlags)]
X_test, y_test = X[(ntrain-nlags):], y[(ntrain-nlags):]

In [ ]:
# LSTM neural network
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(nlags, xdim), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, 'linear'))

model.compile(optimizer='adam', loss='mse')

fit = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.1)
#loss = np.asarray(fit.history['loss'])
plt.plot(fit.history['loss'], label='Training loss')
plt.plot(fit.history['val_loss'], label='Validation loss')
plt.legend()

with open('model.pkl', "wb") as picklefile:
    pickle.dump(model, picklefile)